# Tree Genereration

## Input Selection

In [ ]:
import ipywidgets as widgets
import os, shutil
from IPython.display import clear_output

SAMPLE = None

project_path        = os.path.dirname(os.getcwd())
data_path           = project_path + '/data'
custom_input        = data_path + "/custom_input.txt"
children_file_path  = data_path + "/small.children.txt"
cat_code_file_path  = data_path + "/small.catcode.txt"
word2vec            = data_path + "/glove.txt"
nballs_path         = data_path + "/nballs.txt"

def generate_files(glove_file_path, output_path, sample, input_file_path):
    generate_files_string_command = "main_vis.py --gen --w2v %s --output %s --sample %s --input %s"
    command = generate_files_string_command%(glove_file_path, output_path, sample, input_file_path)     
    %run $command
    check_input_for_nballs()
    
def generate_nballs(glove_file_path):
    generate_balls_string_command = "../nball.py --train_nball %s --w2v %s --ws_child %s --ws_catcode %s --log %s/log.txt"
    command = generate_balls_string_command%(nballs_path, glove_file_path, children_file_path, cat_code_file_path, data_path)     
    %run $command
    check_input_for_vis()
    
def visualize_balls():
    vis_string_command = "main_vis.py --vis --balls %s --children"
    command = vis_string_command%(nballs_path, children_file_path)     
    %run $command
    
def write_data_to_file(file_path, lines):
    with open(file_path, 'w') as file:
        for line in lines:
            file.write(line)
            if not '\n' in line:
                file.write("\n")

# UI Widgets
input_examples = widgets.Combobox( placeholder='Choose Example', description='Input:',
    options=['Cities','Fruits','Small','Medium','Large','Custom']
)

check_input_button     = widgets.Button(description='Check', button_style='success')
words_input            = widgets.Textarea(value='',description='Words', placeholder='keywords comma separated')
word2vec_path_input    = widgets.Text(placeholder='word2vec', value = word2vec, description='word2vec')
generate_tree_button   = widgets.Button(description='Generate Tree', button_style='success')
generate_nballs_button = widgets.Button(description='Generate Balls', button_style='success')
vis_button             = widgets.Button(description='Vis', button_style='success')
buttons_box            = widgets.Box([word2vec_path_input,generate_tree_button])
custom_box             = widgets.Box([words_input,check_input_button])


# UI Widget functions
def on_input_example_choose(change):
    global SAMPLE
    if change['type'] == 'change' and change['name'] == 'value':
        choosen_values = change['new']
        input_examples.disabled = True
        SAMPLE = choosen_values
        if (choosen_values == 'Custom'):
            display(custom_box)
            SAMPLE = None
        else:
            display(buttons_box)

def on_check_button_clicked(button):
    parsed_tokens = [x.strip() for x in str(words_input.value).split(',')]
    words = [x for x in parsed_tokens if x]
    if len(words) > 0:
        words_input.disabled = True
        check_cutom_input_button.disabled = True
        write_data_to_file(custom_input,words)
        display(buttons_box)
    else:
        print("Can't parse your input! Try Again!")
        
def check_input_for_nballs():
    if os.path.exists(children_file_path) and os.path.exists(cat_code_file_path):
        display(generate_nballs_button)
    
def check_input_for_vis():
    if os.path.exists(nballs_path):
        display(vis_button)
    
def on_generate_tree_button_clicked(button):
    if word2vec_path_input.value:
        word2vec_path_input.disabled = True
        generate_tree_button.disabled = True
        generate_files(word2vec_path_input.value, data_path, SAMPLE, custom_input)
        
def on_generate_balls_button_clicked(button):
    print("Start Generating balls")
    generate_nballs_button.disabled = True
    generate_nballs(word2vec_path_input.value)

def on_vis_button_clicked(button):
    visualize_balls()
    
# Set Widget listeners
input_examples.observe(on_input_example_choose)
check_input_button.on_click(on_check_button_clicked)
generate_tree_button.on_click(on_generate_tree_button_clicked)
generate_nballs_button.on_click(on_generate_balls_button_clicked)
vis_button.on_click(on_vis_button_clicked)

# Start UI nteraction
display(input_examples)

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
from matplotlib.patches import Circle
import matplotlib.animation as animation

import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
import random

from sklearn.decomposition import PCA
from decimal import *
from math import pi, cos, sin

ax = None
fig = None
checkbox_items = []

# UI Widgets
visualize_balls         = widgets.Button(description='Visualize Nball', button_style='success')
clear_button_click      = widgets.Button(description='Clear result'   , button_style='danger' )
select_all_button       = widgets.Button(description='Select All'     , button_style='success')
unselect_all_button     = widgets.Button(description='Unselect All'   , button_style='danger')
nball_input_buttons_box = widgets.Box([visualize_balls,clear_button_click])
select_unselect_all_buttons_box = widgets.Box([select_all_button,unselect_all_button])

def read_balls_file(file_path, circles_dic=None):
    if circles_dic is None:
        circles_dic = dict()
    with open(file_path, mode="r", encoding="utf-8") as balls_file:
        for line in balls_file.readlines():
            tokens = line.strip().split()
            circles_dic[tokens[0]] = [float(ele) for ele in tokens[1:]]
        return circles_dic

def create_checkbox_grid(words):
    global ax
    global fig
    global checkbox_items
    for word in words:
        checkbox_changed = widgets.Checkbox(value=True, description=word)
        checkbox_changed.observe(checkbox_changed_callback)
        checkbox_items.append(checkbox_changed)
    grid = widgets.GridBox(checkbox_items, layout=widgets.Layout(grid_template_columns="repeat(5, 150px)"))
    display(select_unselect_all_buttons_box)
    display(grid)
    fig, ax = pyplot.subplots()
    ax.set_aspect(1)
    fig.set_figheight(6)
    fig.set_figwidth(9)

def reduce_and_plot(nball_file_path):
    global recuded_vectors
    global words
    global radius
    
    create_checkbox_grid(words)
    
    
    balls = read_and_parse_balls_file(nball_file_path)
    vectors, words, radius, lengths = get_arrays_from_balls(balls, [], [], True)
    
    recuded_vectors = reduce_dimensions(vectors)
    plot(recuded_vectors, radius, words , set(words))

def update_plot():
    global recuded_vectors
    global words
    global radius
    pyplot.cla()
    showen_words = set()
    for checkbox in checkbox_items:
        if checkbox.value:
            showen_words.add(checkbox.description)
    plot(recuded_vectors,radius, words , showen_words)


def checkbox_changed_callback(b):
    if b['type'] == 'change' and b['name'] == 'value':
        update_plot()


# UI Widget functions
def on_visualize_balls_button_clicked(button):
    nball_file_path = nball_path_input.value
    reduce_and_plot(nball_file_path)
    nball_input_buttons_box.layout.visibility = 'hidden'
        
def on_clear_button_clicked(button):
    clear_output()
    start()

def on_select_all_button_clicked(button):
    for checkbox in checkbox_items:
        checkbox.unobserve_all()
        checkbox.value = True
        checkbox.observe(checkbox_changed_callback)
    update_plot()
    

def on_unselect_all_button_clicked(button):
    for checkbox in checkbox_items:
        checkbox.unobserve_all()
        checkbox.value = False
        checkbox.observe(checkbox_changed_callback)
    update_plot()
        
    
# Set Widget listeners
visualize_balls.on_click(on_visualize_balls_button_clicked)
clear_button_click.on_click(on_clear_button_clicked)
select_all_button.on_click(on_select_all_button_clicked)
unselect_all_button.on_click(on_unselect_all_button_clicked)

    
# Start UI Interaction
def start():
    
start()